<a href="https://colab.research.google.com/github/Arthur-ca/MIE1517/blob/YayunYang/Project_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
import random
import torchvision
from torchvision import datasets, models
from collections import defaultdict
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as F1
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torchvision.datasets import ImageFolder
import cv2 as cv
import tensorflow as tf
import gc
from collections import Counter
import imgaug.augmenters as iaa

In [ ]:
directory_path = '/content/gdrive/MyDrive/archive'
ds_store_path = os.path.join(directory_path, '.DS_Store')

if os.path.exists(ds_store_path):
    os.remove(ds_store_path)
else:
    print(".DS_Store file does not exist in the directory.")

.DS_Store file does not exist in the directory.


In [ ]:
#First link it to google drive and check the subfolder of the data
from google.colab import drive
drive.mount('/content/gdrive')
drive_content = os.listdir('/content/gdrive/MyDrive/archive')
print(drive_content)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
classeses = []
for i in os.listdir('/content/gdrive/MyDrive/archive'):
    if i!=5:
        classeses.append(i)
classeses

['surprise',
 'neutrality',
 'fear',
 'happiness',
 'sadness',
 'contempt',
 'anger',
 'disgust']

In [ ]:
# def gauss_noise_tensor(img):
#     assert isinstance(img, torch.Tensor)
#     dtype = img.dtype
#     if not img.is_floating_point():
#         img = img.to(torch.float32)

#     sigma = 25.0

#     out = img + sigma * torch.randn_like(img)

#     if out.dtype != dtype:
#         out = out.to(dtype)

#     return out
# class ImgAugTransform:
#     def __init__(self):
#         self.aug = iaa.Sequential([
#             iaa.Resize({"height": 224, "width": 224}),
#             iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 3.0))),
#             iaa.Sometimes(0.25, iaa.Affine(rotate=(-20, 20), mode='symmetric')),
#             iaa.Sometimes(0.25, iaa.OneOf([
#                 iaa.Dropout(p=(0, 0.1)),
#                 iaa.CoarseDropout(0.1, size_percent=0.5)
#             ])),
#             iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)
#         ])

#     def __call__(self, img):
#         img = np.array(img)

#         augmented_img = self.aug.augment_image(img)

#         # Convert the augmented image to PyTorch tensor
#         augmented_tensor = F1.to_tensor(augmented_img)
#         return self.aug.augment_image(img)
#   imgaug_transform = ImgAugTransform()

In [ ]:
features_or = []
labels_or = []
train_data_or = []
data_transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
def create_train2(data_percentage=5):
    for i, photo in enumerate(classeses):
        path = os.path.join('/content/gdrive/MyDrive/archive', photo)
        label = i  # Use the index of the class as the label

        # List all images in the class folder
        img_list = os.listdir(path)

        # Calculate the number of images to load (5% of total)
        num_images_to_load = int(len(img_list) * (data_percentage / 100))

        # Randomly select a subset of images
        selected_images = random.sample(img_list, num_images_to_load)


        for img_name in selected_images:
            img_path = os.path.join(path, img_name)
            img_pil = Image.open(img_path)
            transformed_image1 = data_transform1(img_pil)

            if img_pil is None:
                print("Error loading image:", img_path)
                continue

            # Append the original image and its label to the list
            features_or.append(transformed_image1)
            labels_or.append(label)

            train_data_or.append([img_pil, label])

create_train2(data_percentage=5)  # Load only 5% of the data
print(type(features_or[0]))
# Count the occurrences of each label
label_counts_or = Counter(labels_or)

# Print the label counts
for label, count in label_counts_or.items():
    print(f"Label {label}: {count} images")


<class 'torch.Tensor'>
Label 0: 38 images
Label 1: 26 images
Label 2: 28 images
Label 3: 70 images
Label 4: 37 images
Label 5: 10 images
Label 6: 44 images
Label 7: 21 images


In [ ]:
# Define a transformation to convert PIL images to PyTorch tensors
data_transform_to_tensor = transforms.Compose([
    transforms.ToTensor()
])

# Convert images and labels to PyTorch tensors
train_images = torch.stack([data_transform_to_tensor(data[0]) for data in train_data_or])
train_labels = torch.tensor([data[1] for data in train_data_or])

# Assuming you have a DataLoader for your training loop
train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
def get_accuracy(model, dataset):
    correct = 0
    total = 0
    for imgs, labels in torch.utils.data.DataLoader(dataset, batch_size=64):
        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        #############################################
        output = model(imgs)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
def trainSmall(model, data, batch_size=64, learning_rate=0.01, num_epochs=500):
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_acc = []

    # training
    print("Training Start")
    n = 0  # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):
            #############################################
            # To Enable GPU Usage
            imgs = imgs.cuda()
            labels = labels.cuda()
            #############################################

            # Check if imgs is already a tensor
            if not torch.is_tensor(imgs):
                img_to_tensor = transforms.ToTensor()
                imgs = img_to_tensor(imgs)

                # Convert input to float
                imgs = imgs.float()

            # Convert the input tensor to the desired type
            imgs = imgs.type(torch.float32)

            out = model(imgs)  # forward pass
            loss = criterion(out, labels)  # compute the total loss
            loss.backward()  # backward pass (compute parameter updates)
            optimizer.step()  # make the updates for each parameter
            optimizer.zero_grad()  # a clean-up step for PyTorch
            n += 1

        train_accuracy = get_accuracy(model, data)
        train_acc.append(train_accuracy)
        print("Epoch:{}, Accuracy:{}".format(epoch, train_accuracy))
    return train_acc

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNLargeNet2(nn.Module):
    def __init__( self, output1=32, output2=64, output3=128, output4=256):
        super(CNNLargeNet2, self).__init__()
        self.name = "CNN"

        # Define the sequential layers for feature extraction
        self.features = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 5
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        # Flatten the tensor for the fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 1024),
            nn.ReLU(inplace=True),
            #nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            #nn.Dropout(p=0.5),
            nn.Linear(512, 8)
        )

    def forward(self, x):
        x = self.features(x)  # Pass the input through the feature layers
        x = x.view(x.size(0), -1)  # Flatten the tensor for the classifier
        x = self.classifier(x)  # Pass the flattened tensor through the classifier
        return x


In [ ]:
use_cuda = True
if torch.cuda.is_available():
    model = CNNLargeNet2().cuda()
Y = trainSmall(model, train_dataset)

Training Start
Epoch:0, Accuracy:0.07664233576642336
Epoch:1, Accuracy:0.16058394160583941
Epoch:2, Accuracy:0.22992700729927007
Epoch:3, Accuracy:0.08394160583941605
Epoch:4, Accuracy:0.2664233576642336
Epoch:5, Accuracy:0.2518248175182482
Epoch:6, Accuracy:0.2518248175182482
Epoch:7, Accuracy:0.26277372262773724
Epoch:8, Accuracy:0.2737226277372263
Epoch:9, Accuracy:0.2846715328467153
Epoch:10, Accuracy:0.2846715328467153
Epoch:11, Accuracy:0.28832116788321166
Epoch:12, Accuracy:0.2591240875912409
Epoch:13, Accuracy:0.2664233576642336
Epoch:14, Accuracy:0.2664233576642336
Epoch:15, Accuracy:0.27007299270072993
Epoch:16, Accuracy:0.28102189781021897
Epoch:17, Accuracy:0.2737226277372263
Epoch:18, Accuracy:0.28832116788321166
Epoch:19, Accuracy:0.2846715328467153
Epoch:20, Accuracy:0.28102189781021897
Epoch:21, Accuracy:0.2846715328467153
Epoch:22, Accuracy:0.3029197080291971
Epoch:23, Accuracy:0.291970802919708
Epoch:24, Accuracy:0.2956204379562044
Epoch:25, Accuracy:0.299270072992700

In [ ]:
use_cuda = True
if torch.cuda.is_available():
    model = CNNLargeNet2().cuda()
Y = trainSmall(model, train_dataset, batch_size=64, learning_rate=0.005, num_epochs=260)

Training Start
Epoch:0, Accuracy:0.07664233576642336
Epoch:1, Accuracy:0.25547445255474455
Epoch:2, Accuracy:0.07664233576642336
Epoch:3, Accuracy:0.24817518248175183
Epoch:4, Accuracy:0.2664233576642336
Epoch:5, Accuracy:0.27007299270072993
Epoch:6, Accuracy:0.26277372262773724
Epoch:7, Accuracy:0.27007299270072993
Epoch:8, Accuracy:0.29927007299270075
Epoch:9, Accuracy:0.30656934306569344
Epoch:10, Accuracy:0.31386861313868614
Epoch:11, Accuracy:0.3029197080291971
Epoch:12, Accuracy:0.29927007299270075
Epoch:13, Accuracy:0.3102189781021898
Epoch:14, Accuracy:0.3102189781021898
Epoch:15, Accuracy:0.31386861313868614
Epoch:16, Accuracy:0.291970802919708
Epoch:17, Accuracy:0.3248175182481752
Epoch:18, Accuracy:0.30656934306569344
Epoch:19, Accuracy:0.31386861313868614
Epoch:20, Accuracy:0.3029197080291971
Epoch:21, Accuracy:0.291970802919708
Epoch:22, Accuracy:0.29927007299270075
Epoch:23, Accuracy:0.3029197080291971
Epoch:24, Accuracy:0.3175182481751825
Epoch:25, Accuracy:0.32481751824